In [40]:
import requests
import subprocess
import sys
import os
from urllib.request import urlretrieve
from zipfile import ZipFile

# # Function to download the file from Google Drive
# def download_file_from_google_drive(file_id, destination):
#     url = f"https://drive.google.com/uc?export=download&id={file_id}"
#     response = requests.get(url)
#     if response.status_code == 200:
#         with open(destination, 'wb') as f:
#             f.write(response.content)
#         print(f"{destination} downloaded successfully.")
#     else:
#         print(f"Failed to download {destination}.")

# # Download the requirements.txt file from Google Drive
# file_id = "1e8vmGO8Z9g-QoDW9N5u9jACtx_qCVuM5"
# destination = "requirements.txt"
# download_file_from_google_drive(file_id, destination)

# Function to install missing packages
def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Read the requirements.txt file and store libraries with their versions
libraries = {}

with open('requirements.txt', 'r') as f:
    for line in f:
        # Each line in the file is in the form 'library==version'
        if '==' in line:
            lib, version = line.strip().split('==')
            libraries[lib] = version

# Standard libraries that do not require installation
standard_libraries = [
    'os', 'zipfile', 'urllib', 'csv', 'math'
]

# Check third-party libraries and install them if missing
for lib, version in libraries.items():
    try:
        __import__(lib)
        print(f"{lib} is already installed.")
    except ImportError:
        print(f"{lib} not found. Installing...")
        install_package(f"{lib}=={version}")

# Confirm standard libraries are available (they should be, as they are part of Python)
for lib in standard_libraries:
    try:
        __import__(lib)
        print(f"Standard library {lib} is available.")
    except ImportError:
        print(f"Standard library {lib} not found, but it should be included in the Python standard library.")

# Import the third-party libraries with shorthand notations
import requests
import pandas as pd
import numpy as np
import altair as alt
from vega_datasets import data
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.neighbors import BallTree
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from scipy.stats import t
import urllib.request

requests is already installed.
pandas is already installed.
numpy is already installed.
altair is already installed.
vega_datasets is already installed.
scikit-learn not found. Installing...
seaborn is already installed.
matplotlib is already installed.
statsmodels is already installed.
scipy is already installed.
Standard library os is available.
Standard library zipfile is available.
Standard library urllib is available.
Standard library csv is available.
Standard library math is available.


Add later for the parsing of the COVID dataset

In [41]:
state_code_to_abbr = {
    1: 'AL', 2: 'AK', 4: 'AZ', 5: 'AR', 6: 'CA', 8: 'CO', 9: 'CT', 10: 'DE', 11: 'DC', 12: 'FL', 13: 'GA', 15: 'HI', 16: 'ID', 17: 'IL', 18: 'IN', 19: 'IA', 20: 'KS',
    21: 'KY', 22: 'LA', 23: 'ME', 24: 'MD', 25: 'MA', 26: 'MI', 27: 'MN', 28: 'MS', 29: 'MO', 30: 'MT', 31: 'NE', 32: 'NV', 33: 'NH', 34: 'NJ', 35: 'NM', 36: 'NY',
    37: 'NC', 38: 'ND', 39: 'OH', 40: 'OK', 41: 'OR', 42: 'PA', 44: 'RI', 45: 'SC', 46: 'SD', 47: 'TN', 48: 'TX', 49: 'UT', 50: 'VT', 51: 'VA', 53: 'WA', 54: 'WV', 55: 'WI', 56: 'WY'
    }

def parse_cdc_survey():
    """
    Reads the CDC survey input file and parses the contents using ASCII format.

    :return: CDC survey dataframe.
    
    """

    url='https://drive.google.com/file/d/1g69nPHfxfNtWnKBoq2SODoPzlCClYdhc/view?usp=share-link'
    url='https://drive.usercontent.google.com/download?id={}&export=download&authuser=0&confirm=t'.format(url.split('/')[-2])
    df = pd.read_csv(url)

    return df

def parse_mortality():
    """
    Reads the mortality input file and parses the contents using CSV format.

    :return: Mortality dataframe.
    """
    # df = pd.read_csv('mortality.csv')
    url='https://drive.google.com/file/d/10HPyYhcNzQm4JQp6EpoWrJWEmEO6uVS7/view?usp=share-link'
    url='https://drive.usercontent.google.com/download?id={}&export=download&authuser=0&confirm=t'.format(url.split('/')[-2])
    df = pd.read_csv(url)
    return df

def parse_covid_dataset():
    """
    Reads the COVID input file and parses the contents with additional cleaning steps.

    :return: Cleaned COVID dataset as a pandas dataframe.
    """
    # Step 1: Read the COVID dataset
    # df = pd.read_csv('covid_mortality.csv')
    url='https://drive.google.com/file/d/181jSYrgqjUYc-ba94dl2Lw4Z19vV-ha-/view?usp=share_link'
    url='https://drive.usercontent.google.com/download?id={}&export=download&authuser=0&confirm=t'.format(url.split('/')[-2])
    df = pd.read_csv(url)

    # Step 2: Ensure the relevant columns are present
    relevant_columns = [
        'State', 'Condition Group', 'Condition', 'Age Group',
        'COVID-19 Deaths', 'Number of Mentions', 'Start Date', 'End Date', 'Data As Of', 'Year', 'Month', 'Flag', 'Group'
    ]

    if not all(col in df.columns for col in relevant_columns):
        raise ValueError("One or more required columns are missing from the dataset.")

    # Step 3: Filter out rows where 'state' is missing or irrelevant
    df_filtered = df[df['State'].notna()]

    # Step 4: Clean and standardize state names if necessary (e.g., convert full names to abbreviations)
    df_filtered['State'] = df_filtered['State'].str.upper()

    # Step 5: Handle missing data for 'covid_19_deaths' and 'number_of_mentions'
    df_filtered['COVID-19 Deaths'] = df_filtered['COVID-19 Deaths'].fillna(0).astype(int)
    df_filtered['Number of Mentions'] = df_filtered['Number of Mentions'].fillna(0).astype(int)

    # Step 6: Remove rows where the state is 'UNITED STATES'
    df_filtered = df_filtered[df_filtered['State'] != 'UNITED STATES']

    # Step 7: Drop irrelevant columns
    df_filtered = df_filtered.drop(columns=['Data As Of', 'Year', 'Month', 'Flag', 'Number of Mentions'])

    # Step 8: Ensure 'start_date' and 'end_date' are in datetime format
    df_filtered['Start Date'] = pd.to_datetime(df_filtered['Start Date'])
    df_filtered['End Date'] = pd.to_datetime(df_filtered['End Date'])

    # Step 9: Calculate the time difference in days, months, and years
    days_difference = (df_filtered['End Date'] - df_filtered['Start Date']).dt.days
    months_difference = days_difference / 30.44
    years_difference = days_difference / 365.25  # Accounts for leap years

    # Step 10: Create new columns for daily, monthly, and yearly averages of covid_19_deaths
    df_filtered['daily_avg'] = df_filtered['COVID-19 Deaths'] / days_difference
    df_filtered['monthly_avg'] = df_filtered['COVID-19 Deaths'] / months_difference
    df_filtered['yearly_avg'] = df_filtered['COVID-19 Deaths'] / years_difference

    # Step 11: Drop rows where 'age_group' is 'Not stated' or 'All Ages'
    df_filtered = df_filtered[~df_filtered['Age Group'].isin(['Not stated', 'All Ages'])]

    # Step 12: Drop rows where the condition is 'COVID-19' (as we focus on pre-existing conditions)
    df_filtered = df_filtered[df_filtered['Condition'] != 'COVID-19']

    # Step 13: Drop the columns 'start_date', 'end_date', 'covid_19_deaths', and 'group'
    df_filtered = df_filtered.drop(columns=['Start Date', 'End Date', 'COVID-19 Deaths', 'Group'])

    # Step 14: Reset the index
    df_filtered.reset_index(drop=True, inplace=True)

    return df_filtered

Loading of the Parsed BRFSS Survey

In [42]:
df_cdc_survey = parse_cdc_survey()
df_cdc_survey.head(20)

C:\Users\natal\AppData\Local\Temp\ipykernel_29060\1992358871.py:17: DtypeWarning: Columns (19,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url)


,State Abbr.,Interview Year,Is Adult Landline,Gender,Num of Adults Landline,General Health,Could Afford Doctor,Years Since Last Checkup,Exercise in Past 30 Days,Hours of Sleeping,...,BMI Category,Income Level,Smoking,Cigarettes per Day,Drinks in Last 30 Days,Prediabetes,Years Since Last Dentist Visit,Is Adult Cell,Num of Adults Cell,Shortness of Breath
0,AL,2022,True,Female,2.0,Very good,False,within_past_year,False,8.0,...,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,True
1,AL,2022,True,Female,2.0,Excellent,False,NaN,False,6.0,...,over_weight,between_25K_and_35K,NaN,NaN,NaN,True,NaN,NaN,NaN,True
2,AL,2022,True,Female,1.0,Very good,False,within_past_year,True,5.0,...,over_weight,between_100K_and_200K,NaN,NaN,NaN,True,NaN,NaN,NaN,True
3,AL,2022,True,Female,3.0,Excellent,False,within_past_year,True,7.0,...,normal_weight,NaN,some_days,2.0,NaN,True,NaN,NaN,NaN,True
4,AL,2022,True,Female,2.0,Fair,False,within_past_year,True,9.0,...,normal_weight,between_25K_and_35K,NaN,NaN,2.0,True,NaN,NaN,NaN,True
5,AL,2022,True,Male,1.0,Poor,False,within_past_year,False,7.0,...,over_weight,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,True
6,AL,2022,True,Female,1.0,Very good,False,within_past_year,True,7.0,...,normal_weight,between_50K_and_100K,never,35.0,2.0,True,NaN,NaN,NaN,True
7,AL,2022,True,Female,1.0,Good,False,within_past_year,False,8.0,...,over_weight,between_50K_and_100K,NaN,NaN,NaN,True,NaN,NaN,NaN,True
8,AL,2022,True,Female,2.0,Good,False,within_past_year,True,6.0,...,NaN,between_50K_and_100K,never,5.0,NaN,True,NaN,NaN,NaN,True
9,AL,2022,True,Female,2.0,Good,False,within_past_year,True,7.0,...,over_weight,between_50K_and_100K,NaN,NaN,1.0,True,NaN,NaN,NaN,True


Loading of the Parsed Mortality Dataset

In [43]:
df_mortality = parse_mortality()
df_mortality.head()

,Year,LocationAbbr,LocationDesc,GeographicLevel,DataSource,Class,Topic,Data_Value,Data_Value_Unit,Data_Value_Type,Data_Value_Footnote_Symbol,Data_Value_Footnote,StratificationCategory1,Stratification1,StratificationCategory2,Stratification2,TopicID,LocationID,Y_lat,X_lon
0,2019,AK,Aleutians East,County,NVSS,Cardiovascular Diseases,Heart Disease Mortality,182.4,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",NaN,NaN,Gender,Overall,Race/Ethnicity,Overall,T2,2013,55.442393,-161.959930
1,2019,AK,Aleutians West,County,NVSS,Cardiovascular Diseases,Heart Disease Mortality,172.6,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",NaN,NaN,Gender,Overall,Race/Ethnicity,Overall,T2,2016,53.648343,-166.917533
2,2019,AK,Anchorage,County,NVSS,Cardiovascular Diseases,Heart Disease Mortality,255.6,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",NaN,NaN,Gender,Overall,Race/Ethnicity,Overall,T2,2020,61.159145,-149.104559
3,2019,AK,Bethel,County,NVSS,Cardiovascular Diseases,Heart Disease Mortality,343.4,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",NaN,NaN,Gender,Overall,Race/Ethnicity,Overall,T2,2050,60.923648,-159.753262
4,2019,AK,Bristol Bay,County,NVSS,Cardiovascular Diseases,Heart Disease Mortality,NaN,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",~,Insufficient Data,Gender,Overall,Race/Ethnicity,Overall,T2,2060,58.753496,-156.694662


Mortality dataset has statistics at both state and county levels. We will first focus our efforts on the counties.

25841 values (44.5%) from the data_value are missing in the mortality dataset at county level. We need imputation to avoid losing valuable information.

In [44]:
df_mortality_county = df_mortality[df_mortality['GeographicLevel'] == 'County'].copy()

#Commented out for conciseness. Uncomment to see number of missing values per column.
#df_mortality_county.isnull().sum()

We will impute missing data by averaging the values from four closest neigbors. We first convert the lat and lon coordinates into radians because we will use those shortly for imputation.

In [45]:
for col in df_mortality_county[["Y_lat", "X_lon"]]:
    rad = np.deg2rad(df_mortality_county[col].values)
    df_mortality_county[f'{col}_rad'] = rad

df_mortality_county_without_nan = df_mortality_county[df_mortality_county['Data_Value'].notna()].copy()

Ball tree is a data structure that can handle geospatial data for efficient search. With the help of a ball tree, we can reduce the search time for all missing data from
$O(n^2)$ down to
$O(nlogn)$

Ball trees work on radian data and that is why we did that conversion above.

In [46]:
ball = BallTree(df_mortality_county_without_nan[['Y_lat_rad', 'X_lon_rad']].values, metric='haversine')

The function below finds the four nearest neigbour counties and averages the data from them.

In [47]:
def impute_from_neigbours(row):
    """
    Imputes missing information from the closest neigbors.

    :param row: Current row of the dataset to be processed.
    """
    if np.isnan(row['Data_Value']):
        _, indices = ball.query([row[['Y_lat_rad', 'X_lon_rad']].values], k=4)
        row['Data_Value'] = df_mortality_county_without_nan.iloc[indices[0]]['Data_Value'].mean()
    return row

In [48]:
df_mortality_county_imputed = df_mortality_county.apply(impute_from_neigbours, axis=1)

In [49]:
df_covid = parse_covid_dataset()
df_covid.head()

,State,Condition Group,Condition,ICD10_codes,Age Group,daily_avg,monthly_avg,yearly_avg
0,ALABAMA,Respiratory diseases,Influenza and pneumonia,J09-J18,0-24,0.014695,0.447318,5.367377
1,ALABAMA,Respiratory diseases,Influenza and pneumonia,J09-J18,25-34,0.075680,2.303688,27.641991
2,ALABAMA,Respiratory diseases,Influenza and pneumonia,J09-J18,35-44,0.169728,5.166525,61.993204
3,ALABAMA,Respiratory diseases,Influenza and pneumonia,J09-J18,45-54,0.402645,12.256517,147.066128
4,ALABAMA,Respiratory diseases,Influenza and pneumonia,J09-J18,55-64,0.876561,26.682528,320.164034


**New York City is treated as a seperate entitiy, for the time being lets leave it to see if we can explore Metro differences**

In [50]:
df_cdc_survey.shape

(344751, 38)

In [51]:
df_mortality.shape

(59094, 20)

In [52]:
df_covid.shape

(466400, 8)

In [53]:
print(df_cdc_survey['State Abbr.'].unique())
print(df_mortality['LocationAbbr'].unique())
print(df_covid['State'].unique())


['AL' 'AK' 'AZ' 'AR' 'CA' 'CO' 'CT' 'DE' 'DC' 'FL' 'GA' 'HI' 'ID' 'IL'
 'IN' 'IA' 'KS' 'KY' 'LA' 'ME' 'MD' 'MA' 'MI' 'MN' 'MS' 'MO' 'MT' 'NE'
 'NV' 'NH' 'NJ' 'NM' 'NY' 'NC' 'ND' 'OH' 'OK' 'OR' 'PA' 'RI' 'SC' 'SD'
 'TN' 'TX' 'UT' 'VT' 'VA' 'WA' 'WV' 'WI' 'WY']
['AK' 'AL' 'AR' 'AS' 'AZ' 'CA' 'CO' 'CT' 'DC' 'DE' 'FL' 'GA' 'GU' 'HI'
 'IA' 'ID' 'IL' 'IN' 'KS' 'KY' 'LA' 'MA' 'MD' 'ME' 'MI' 'MN' 'MO' 'MP'
 'MS' 'MT' 'NC' 'ND' 'NE' 'NH' 'NJ' 'NM' 'NV' 'NY' 'OH' 'OK' 'OR' 'PA'
 'PR' 'RI' 'SC' 'SD' 'TN' 'TX' 'US' 'UT' 'VA' 'VI' 'VT' 'WA' 'WI' 'WV'
 'WY']
['ALABAMA' 'ALASKA' 'ARIZONA' 'ARKANSAS' 'CALIFORNIA' 'COLORADO'
 'CONNECTICUT' 'DELAWARE' 'DISTRICT OF COLUMBIA' 'FLORIDA' 'GEORGIA'
 'HAWAII' 'IDAHO' 'ILLINOIS' 'INDIANA' 'IOWA' 'KANSAS' 'KENTUCKY'
 'LOUISIANA' 'MAINE' 'MARYLAND' 'MASSACHUSETTS' 'MICHIGAN' 'MINNESOTA'
 'MISSISSIPPI' 'MISSOURI' 'MONTANA' 'NEBRASKA' 'NEVADA' 'NEW HAMPSHIRE'
 'NEW JERSEY' 'NEW MEXICO' 'NEW YORK' 'NEW YORK CITY' 'NORTH CAROLINA'
 'NORTH DAKOTA' 'OHIO' 'OKLAHOMA' '

In [54]:
# Dictionary to map state names to abbreviations
state_name_to_abbr = {
    'ALABAMA': 'AL', 'ALASKA': 'AK', 'ARIZONA': 'AZ', 'ARKANSAS': 'AR', 
    'CALIFORNIA': 'CA', 'COLORADO': 'CO', 'CONNECTICUT': 'CT', 'DELAWARE': 'DE', 
    'DISTRICT OF COLUMBIA': 'DC', 'FLORIDA': 'FL', 'GEORGIA': 'GA', 'HAWAII': 'HI', 
    'IDAHO': 'ID', 'ILLINOIS': 'IL', 'INDIANA': 'IN', 'IOWA': 'IA', 'KANSAS': 'KS', 
    'KENTUCKY': 'KY', 'LOUISIANA': 'LA', 'MAINE': 'ME', 'MARYLAND': 'MD', 
    'MASSACHUSETTS': 'MA', 'MICHIGAN': 'MI', 'MINNESOTA': 'MN', 'MISSISSIPPI': 'MS', 
    'MISSOURI': 'MO', 'MONTANA': 'MT', 'NEBRASKA': 'NE', 'NEVADA': 'NV', 
    'NEW HAMPSHIRE': 'NH', 'NEW JERSEY': 'NJ', 'NEW MEXICO': 'NM', 'NEW YORK': 'NY', 
    'NEW YORK CITY': 'NYC', 'NORTH CAROLINA': 'NC', 'NORTH DAKOTA': 'ND', 'OHIO': 'OH', 
    'OKLAHOMA': 'OK', 'OREGON': 'OR', 'PENNSYLVANIA': 'PA', 'RHODE ISLAND': 'RI', 
    'SOUTH CAROLINA': 'SC', 'SOUTH DAKOTA': 'SD', 'TENNESSEE': 'TN', 'TEXAS': 'TX', 
    'UTAH': 'UT', 'VERMONT': 'VT', 'VIRGINIA': 'VA', 'WASHINGTON': 'WA', 
    'WEST VIRGINIA': 'WV', 'WISCONSIN': 'WI', 'WYOMING': 'WY', 'PUERTO RICO': 'PR'
}

# Map state names to abbreviations in df_covid
df_covid['state_abbr'] = df_covid['State'].map(state_name_to_abbr)

print(df_covid[['State', 'state_abbr']].head())


     State state_abbr
0  ALABAMA         AL
1  ALABAMA         AL
2  ALABAMA         AL
3  ALABAMA         AL
4  ALABAMA         AL


In [55]:
df_covid.head(5)

,State,Condition Group,Condition,ICD10_codes,Age Group,daily_avg,monthly_avg,yearly_avg,state_abbr
0,ALABAMA,Respiratory diseases,Influenza and pneumonia,J09-J18,0-24,0.014695,0.447318,5.367377,AL
1,ALABAMA,Respiratory diseases,Influenza and pneumonia,J09-J18,25-34,0.075680,2.303688,27.641991,AL
2,ALABAMA,Respiratory diseases,Influenza and pneumonia,J09-J18,35-44,0.169728,5.166525,61.993204,AL
3,ALABAMA,Respiratory diseases,Influenza and pneumonia,J09-J18,45-54,0.402645,12.256517,147.066128,AL
4,ALABAMA,Respiratory diseases,Influenza and pneumonia,J09-J18,55-64,0.876561,26.682528,320.164034,AL


In [56]:
print(df_cdc_survey.dtypes)



State Abbr.                        object
Interview Year                      int64
Is Adult Landline                    bool
Gender                             object
Num of Adults Landline            float64
General Health                     object
Could Afford Doctor                  bool
Years Since Last Checkup           object
Exercise in Past 30 Days             bool
Hours of Sleeping                 float64
Heart Attack                         bool
Heart Disease                        bool
Stroke                               bool
Asthma                               bool
Asthma in Effect                     bool
Depression                           bool
Kidney Disease                       bool
Arthritis                            bool
Diabetes                             bool
Life Satisfaction                  object
Ethnicity                          object
Metropolitan                         bool
Age Group                          object
Education                         

In [57]:
print(df_mortality_county_imputed.dtypes)

Year                            int64
LocationAbbr                   object
LocationDesc                   object
GeographicLevel                object
DataSource                     object
Class                          object
Topic                          object
Data_Value                    float64
Data_Value_Unit                object
Data_Value_Type                object
Data_Value_Footnote_Symbol     object
Data_Value_Footnote            object
StratificationCategory1        object
Stratification1                object
StratificationCategory2        object
Stratification2                object
TopicID                        object
LocationID                      int64
Y_lat                         float64
X_lon                         float64
Y_lat_rad                     float64
X_lon_rad                     float64
dtype: object


In [58]:
print(df_covid.dtypes)

State               object
Condition Group     object
Condition           object
ICD10_codes         object
Age Group           object
daily_avg          float64
monthly_avg        float64
yearly_avg         float64
state_abbr          object
dtype: object


In [59]:
bmi_values = df_cdc_survey['BMI Category'].unique()
print(bmi_values)

[nan 'over_weight' 'normal_weight' 'obese' 'underweight']


# Mapping of CDC Survey

In [60]:
# Map General Health
df_cdc_survey['General Health'] = df_cdc_survey['General Health'].map({
    'Excellent': 5, 'Very good': 4, 'Good': 3, 'Fair': 2, 'Poor': 1
})

# Map Smoking 
df_cdc_survey['Smoking'] = df_cdc_survey['Smoking'].map({
    'never': 0,      # Non-smoker
    'some_days': 1,  # Occasional smoker
    'every_day': 2,  # Daily smoker
})

# Map True to 1 and False to 0 for 'Exercise in Past 30 Days'
df_cdc_survey['Exercise in Past 30 Days'] = df_cdc_survey['Exercise in Past 30 Days'].map({
    True: 1,
    False: 0
})

# Map categorical 'Years Since Last Checkup' values to numeric
df_cdc_survey['Years Since Last Checkup'] = df_cdc_survey['Years Since Last Checkup'].map({
    'within_past_year': 1,
    'within_past_two_years': 2,
    'within_past_five_years': 3,
    'five_or_more_years': 5
})

# Map categorical BMI values to numeric
df_cdc_survey['BMI Category'] = df_cdc_survey['BMI Category'].map({
    'underweight': 0,
    'normal_weight': 1,
    'over_weight': 2,
    'obese': 3
})

# Replace NaN values in 'Cigarettes per Day' and 'Drinks in Last 30 Days' with 0
# This is so we can get a average
df_cdc_survey['Cigarettes per Day'] = df_cdc_survey['Cigarettes per Day'].fillna(0)
df_cdc_survey['Drinks in Last 30 Days'] = df_cdc_survey['Drinks in Last 30 Days'].fillna(0)

# This is the unsupervised agg for clustering on the state level

In [61]:
# Aggregating df_cdc_survey
df_cdc_survey_agg_unsup = df_cdc_survey.groupby('State Abbr.').agg({
    'General Health': 'mean', 
    'Exercise in Past 30 Days': 'mean', 
    'Smoking': 'mean', 
    'Shortness of Breath': 'sum', 
    'Hours of Sleeping': 'mean',
    'BMI Category': 'mean',
    'Years Since Last Checkup': 'mean',
    'Cigarettes per Day': 'mean',
    'Drinks in Last 30 Days': 'mean'
}).reset_index()


In [62]:
# Aggregating df_mortality_county_imputed
df_mortality_agg = df_mortality_county_imputed.groupby('LocationAbbr').agg({
    'Data_Value': 'mean',  # Average heart disease mortality rate in a year
}).reset_index()

# Display the result
print(df_mortality_agg.head())

  LocationAbbr  Data_Value
0           AK  277.438075
1           AL  466.494735
2           AR  445.403111
3           AS   83.631944
4           AZ  282.046111


In [63]:
# Aggregating df_covid by state
df_covid_agg = df_covid.groupby('state_abbr').agg({
    'yearly_avg': 'mean'  # Average yearly cases
}).reset_index()

# Display the result
print(df_covid_agg.head())

  state_abbr  yearly_avg
0         AK    1.527266
1         AL   66.900089
2         AR   41.932062
3         AZ  115.831893
4         CA  538.285840


In [64]:
# Merge df_cdc_survey_agg_unsup with df_mortality_agg_unsup
df_unsup = df_cdc_survey_agg_unsup.merge(df_mortality_agg, left_on='State Abbr.', right_on='LocationAbbr', how='inner')

# Merge the result with df_covid_agg_unsup
df_unsup = df_unsup.merge(df_covid_agg, left_on='State Abbr.', right_on='state_abbr', how='inner')

# Drop duplicate columns for state
df_unsup.drop(columns=['LocationAbbr', 'state_abbr'], inplace=True)

df_unsup.head(5)


,State Abbr.,General Health,Exercise in Past 30 Days,Smoking,Shortness of Breath,Hours of Sleeping,BMI Category,Years Since Last Checkup,Cigarettes per Day,Drinks in Last 30 Days,Data_Value,yearly_avg
0,AK,3.456126,0.780876,0.553381,5020,7.054147,2.005106,1.578323,5.894223,1.194223,277.438075,1.527266
1,AL,3.208700,0.687758,0.592010,3872,6.998162,2.133351,1.262841,5.818182,0.817665,466.494735,66.900089
2,AR,3.159434,0.662369,0.614639,4111,7.052125,2.082006,1.296665,7.329847,0.940161,445.403111,41.932062
3,AZ,3.370771,0.755157,0.486242,7417,7.087786,1.984653,1.434741,5.742618,1.091142,282.046111,115.831893
4,CA,3.454979,0.800115,0.422476,6949,7.012274,1.907335,1.474276,4.070514,1.214563,289.406442,538.285840
